In [21]:
import os
from os import walk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim
from torch.autograd import Variable
from progressbar import ProgressBar
from tqdm import tqdm
import time
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet")

In [30]:
class Model_Loader(object):
    def __init__(self, model, optimizer, device=None):
        if device == None:
            raise ValueError(
                r"please indicate device by running device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')")
        self.device = device
        self.model = model.to(self.device)
        self.best_model = model.to(self.device)
        self.optimizer = optimizer
        self.loss_list = []
        self.min_loss = float("inf")

    def fit(self, dataloader, loss_function, epoch_num, save_path=None):
        print("Learning Rate is :", self.optimizer.state_dict()['param_groups'][0]["lr"])
        loss_function = loss_function.to(self.device)

        for epoch in tqdm(range(epoch_num)):
            total_loss = 0
            for _, (inputs, outputs) in enumerate(dataloader):
                inputs = Variable(inputs).float().to(self.device)
                print(inputs.shape)
                outputs = Variable(outputs).float().to(self.device)
                self.optimizer.zero_grad()  # noticed:  the grad return to zero before starting the loop
                
                # forward + backward +update
                pred = self.model(inputs)
                pred = pred.to(self.device)
                loss = loss_function(pred, outputs)
                loss.backward()
                self.optimizer.step()

                total_loss += loss.item()
            total_loss = total_loss * dataloader.batch_size / dataloader.dataset.tensors[0].shape[0]
            print('Epoch: ', epoch + 1, ' loss: ', total_loss)
            self.loss_list.append(total_loss)
            if save_path is not None:
                np.save(save_path + "loss.npy", self.loss_list)
                if total_loss < self.min_loss:
                    self.best_model = self.model
                    torch.save(self.model, save_path + "best_model.tar")
                    self.min_loss = total_loss
        if save_path is not None:
            torch.save(self.model, save_path + "model.tar")
            plt.plot(self.loss_list, color='r')
            plt.savefig(save_path + "loss.png")
        return self.model

    def transform(self, dataloader):
        pred_list = []
        label_list = []
        for _, (data, label) in enumerate(dataloader):
            data = Variable(data).float().to(self.device)
            pred = self.model(data).to(self.device)
            pred_list.extend(pred.tolist())
            label_list.extend(label.tolist())
        self.testy_pred = pd.DataFrame(pred_list)
        self.testy = pd.DataFrame(label_list)
        return self.testy_pred

In [31]:
x = torch.randn(5000,108,3)
y = torch.randn(5000,1)

In [32]:
train_dataset = Data.TensorDataset(x, y)
train_loader = Data.DataLoader(
    dataset=train_dataset,
    batch_size=1024,
    shuffle=True,
    num_workers=16,
    pin_memory=True
)

In [33]:
from AlphaNet.Models import AlphaNet_LSTM_V1
LR = 0.01
epoch_num = 10
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
loss_function = nn.MSELoss()
model = AlphaNet_LSTM_V1(108, 30)
optimizer = optim.Adam(model.parameters(), lr=LR)
model_loader = Model_Loader(model = model,optimizer=optimizer,device=device)
print(model_loader.model)

AlphaNet_LSTM_V1(
  (batch): BatchNorm1d(108, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(108, 30, num_layers=2, batch_first=True, bidirectional=True)
  (batch2): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (out): Linear(in_features=30, out_features=1, bias=True)
)


In [35]:
model = model_loader.fit(train_loader,loss_function = loss_function,epoch_num = epoch_num)

  0%|          | 0/10 [00:00<?, ?it/s]

Learning Rate is : 0.01
torch.Size([1024, 108, 3])
torch.Size([1024, 108, 3])
torch.Size([1024, 108, 3])
torch.Size([1024, 108, 3])
torch.Size([904, 108, 3])


 10%|█         | 1/10 [00:01<00:15,  1.75s/it]

Epoch:  1  loss:  1.2961306396484376
torch.Size([1024, 108, 3])
torch.Size([1024, 108, 3])
torch.Size([1024, 108, 3])
torch.Size([1024, 108, 3])
torch.Size([904, 108, 3])


 20%|██        | 2/10 [00:03<00:13,  1.68s/it]

...

Epoch:  8  loss:  0.38566986694335936
torch.Size([1024, 108, 3])
torch.Size([1024, 108, 3])
torch.Size([1024, 108, 3])
torch.Size([1024, 108, 3])
torch.Size([904, 108, 3])


 90%|█████████ | 9/10 [00:15<00:01,  1.73s/it]

Epoch:  9  loss:  0.2700466278076172
torch.Size([1024, 108, 3])
torch.Size([1024, 108, 3])
torch.Size([1024, 108, 3])
torch.Size([1024, 108, 3])
torch.Size([904, 108, 3])


100%|██████████| 10/10 [00:17<00:00,  1.73s/it]

Epoch:  10  loss:  0.1960443084716797


In [29]:
for _, (inputs, outputs) in enumerate(train_loader):
    inputs = Variable(inputs).float().to("cuda:0")
    print(inputs.shape)

torch.Size([1024, 108, 3])
torch.Size([1024, 108, 3])
torch.Size([1024, 108, 3])
torch.Size([1024, 108, 3])
torch.Size([904, 108, 3])
